[ Soynlp ] 학습 기반 토크나이저
- 품사 태깅, 단어 토큰화 등을 지원하는 단어 토크나이저
- 비비도 학습으로 단어 토큰화 => 데이터에 자주 등장하는 단어들을 단어로 분석
- 내부적으로 단어 점수 표로 동작

In [1]:
# %pip install soynlp

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/416.8 kB ? eta -:--:--
   --- ----------------------------------- 41.0/416.8 kB 653.6 kB/s eta 0:00:01
   ---------- ----------------------------- 112.6/416.8 kB 1.1 MB/s eta 0:00:01
   ------------------ --------------------- 194.6/416.8 kB 1.3 MB/s eta 0:00:01
   ------------------------ --------------- 256.0/416.8 kB 1.4 MB/s eta 0:00:01
   ---------------------------------- ----- 358.4/416.8 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 416.8/416.8 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


[기존 형태소 분석기] : 신조어나 형태소 분석기에 등록되지 않은 단어는 제대로 구분하지 못함

In [2]:
from konlpy.tag import Okt

tokenizer = Okt()

# 형태소 분석 시 매개변수 stem=True, norm=True 설정
print(tokenizer.morphs('에이비식스 이대휘 1월 최애돌 기부 요정입니다'))
print(tokenizer.morphs('에이비식스 이대휘 1월 최애돌 기부 요정입니다', stem=True))  # 어간
print(tokenizer.morphs('에이비식스 이대휘 1월 최애돌 기부 요정입니다', norm=True))  # 정규화

['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '입니다']
['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '이다']
['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '입니다']


[Soynlp] 사용 => 말뭉치 데이터셋으로 학습 진행 후 사용 가능

In [3]:
### 학습 데이터 처리
from soynlp import DoublespaceLineCorpus    # 한 개로 통합된 문서 데이터 분리
from soynlp.word import WordExtractor       # 단어 추출

In [7]:
from urllib.request import urlretrieve

news_url = "https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt"
filename = '../datasets/news_data.txt'

ret = urlretrieve(url=news_url, filename=filename)

In [8]:
### 훈련 데이터 문서 분리
corpus = DoublespaceLineCorpus(filename)
print(f'훈련 데이터 문서 : {len(corpus)}개')

훈련 데이터 문서 : 30091개


In [9]:
### soynlp 학습 진행
word_extractor = WordExtractor()

# 학습 진행하며 단어별 점수 생성
word_extractor.train(corpus)

# 단어별 점수표 추출
word_score_table = word_extractor.extract()

training was done. used memory 1.110 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [ ]:
# 단어별 점수표 확인
for idx, key in enumerate(word_score_table.keys()):
    pass

In [13]:
### 응집 확률(cohesion probability) : 내부 문자열(substring)이 얼마나 응집하여 자주 등장하는지를 판단하는 척도
# - 원리 : 문자열을 문자 단위로 분리, 왼쪽부터 순서대로 문자를 추가
#          각 문자열이 주어졌을 때 그 다음 문자가 나올 확률을 계산 / 누적곱 한 값
# - 값이 높을수록 : 전체 코퍼스에서 이 문자열 시퀀스는 하나의 단어로 등장할 가능성 높음

In [14]:
word_score_table['바'].cohesion_forward

0

In [10]:
word_score_table['바다'].cohesion_forward

0.06393648140409527

In [12]:
word_score_table['바다를'].cohesion_forward

0.07716779358040307

In [11]:
word_score_table['바다에'].cohesion_forward

0.11518621707955429

In [15]:
### soynlp의 L tokenizer
# 띄어쓰기 단위로 나눈 어절 토큰 : L 토큰 + R 토큰
# 예 : '공원에' -> '공원 + 에', '공부하는' -> '공부 + 하는'
# 분리 기준 : 점수가 가장 높은 L 토큰을 찾아내는 원리

In [20]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}

L_tokenizer = LTokenizer(scores=scores)
L_tokenizer.tokenize('국제사회와 우리의 노력들로 범죄를 척결하자', flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

In [25]:
### 최대 점수 토크나이저
# - 띄어쓰기가 되지 않는 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저
# - 띄어쓰기가 되어 있지 않은 문장을 넣어서 점수를 통해 토큰화 된 결과
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
print(maxscore_tokenizer.tokenize('국제사회와우리의노력들로범죄를척결하자'
                            # , flatten=False
                            ))
print(maxscore_tokenizer.tokenize('국제사회 와우 리의노력들로범죄를척결하자'
                            # , flatten=False
                            ))

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']
['국제사회', '와우', '리의', '노력', '들로', '범죄', '를', '척결', '하자']


In [26]:
### soynlp를 이용한 반복되는 문자 정제
# - ㅋㅋ, ㅎㅎ 등의 이모티콘의 경우 불필요하게 연속되는 경우가 많음
# - ㅋㅋ, ㅋㅋㅋ, ㅋㅋㅋㅋ 와 같은 경우를 모두 서로 다른 단어로 처리하는 것은 불필요
# >>> 반복되는 것은 하나로 정규화

In [30]:
from soynlp.normalizer import *

print(emoticon_normalize('앜ㅋㅋㅋ이영화존잼ㅠㅠㅠㅠㅠ', num_repeats=1))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋ그영화존잼ㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋ저영화존잼ㅠㅠㅠㅠㅠㅠ', num_repeats=2))

아ㅋ영화존잼ㅠ
아ㅋㅋ영화존잼ㅠㅠ
아ㅋㅋ영화존잼ㅠㅠ


In [31]:
print(repeat_normalize('와하하하핫', num_repeats=2))
print(repeat_normalize('와하하하하하핫', num_repeats=2))

와하하하핫
와하하핫


In [35]:
# !pip install customized_konlpy

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/881.5 kB ? eta -:--:--
   ---------------------------------------- 10.2/881.5 kB ? eta -:--:--
   - ------------------------------------- 41.0/881.5 kB 495.5 kB/s eta 0:00:02
   ----- -------------------------------- 122.9/881.5 kB 901.1 kB/s eta 0:00:01
   --------- ------------------------------ 204.8/881.5 kB 1.1 MB/s eta 0:00:01
   ------------------- -------------------- 430.1/881.5 kB 1.9 MB/s eta 0:00:01
   --------------------------- ------------ 604.2/881.5 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 881.5/881.5 kB 2.8 MB/s eta 0:00:00


In [39]:
from ckonlpy.tag import Twitter

twitter = Twitter()
twitter.morphs('은경이는 사무실로 갔습니다')

c:\ProgramData\anaconda3\envs\Torch_NLP38\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['은', '경이', '는', '사무실', '로', '갔습니다']

In [40]:
twitter.pos('은경이는 사무실로 갔습니다')

[('은', 'Noun'),
 ('경이', 'Noun'),
 ('는', 'Josa'),
 ('사무실', 'Noun'),
 ('로', 'Josa'),
 ('갔습니다', 'Verb')]

In [41]:
# 형태소분석기에 사전 추가
twitter.add_dictionary('은경이', 'Noun')

In [42]:
twitter.morphs('은경이는 사무실로 갔습니다')    # 추가사항 반영

['은경이', '는', '사무실', '로', '갔습니다']